In [1]:

import torch.nn as nn
import torch
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt


In [2]:
#Read the data into a pandas data frame 
df = pd.read_parquet("de_train.parquet")

In [3]:
#normalizes data between -1 and 1
#returns normalized data and the factors used to normalzie 
def normalize(df):
    min = df.min()
    max=df.max()
    df_normalized = (df - min) / (max-min)
    return df_normalized, min.reset_index(drop=True), max.reset_index(drop=True)


#unnormalize
def unnormalize(normalized_df, min, max):
    return min + normalized_df*(max-min)

In [4]:
############################DATA PREP###############################



####One hot incodes inputs####

# Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Fit and transform the data
cells = df["cell_type"].values.reshape(-1, 1)
hot_cells = encoder.fit_transform(cells)
cell_mapping = encoder.categories_[0]

compounds = df['sm_name'].values.reshape(-1, 1)
hot_compounds = encoder.fit_transform(compounds)
compound_mapping = encoder.categories_[0]

#Puts together inputs
inputs = np.hstack((hot_cells, hot_compounds))
inputs_df = pd.DataFrame(data = inputs)


####Normalizes Outputs####

outputs = df.loc[:, 'A1BG':'ZZEF1']
outputs_norm_df, norm_min, norm_max = normalize(outputs)


####Puts Inputs and Outputs Together####
prepped_df = pd.concat([inputs_df, outputs_norm_df], axis=1)



In [5]:
#Break into training and validation and split inputs from outputs
val, train = train_test_split(prepped_df, train_size=.2, random_state=3)
trainIn_df = train.loc[:, 0:151]
trainOut_df = train.loc[:, 'A1BG':'ZZEF1']
valIn_df = val.loc[:,0:151]
valOut_df = val.loc[:, 'A1BG':'ZZEF1']

#Transforms data frames into tensors
trainIn_t = torch.tensor(trainIn_df.values)
trainOut_t = torch.tensor(trainOut_df.values)
valIn_t = torch.tensor(valIn_df.values)
valOut_t = torch.tensor(valOut_df.values)




In [6]:
#Build each layer of nueral network
#Provide forward method to train network
class Layer1(nn.Module):
    def __init__(self, input_size, hidden1_size, num_classes):
        #Initialzing layers
        super(Layer1, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden1_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden1_size, num_classes)
        self.act_output = nn.Sigmoid()
    #training function
    def forward(self,x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.act_output(out)
        return out
    
#Instantiate class with input, hidden layer, and output size 
model1 = Layer1(152,1024,18211)
#Change to double data type
model1.double()

Layer1(
  (fc1): Linear(in_features=152, out_features=1024, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=1024, out_features=18211, bias=True)
  (act_output): Sigmoid()
)

In [7]:
#Build each layer of nueral network
#Provide forward method to train network
class Layer2(nn.Module):
    def __init__(self, input_size, hidden1_size, hidden2_size, num_classes):
        #Initialzing layers
        super(Layer2, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden1_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden1_size, hidden2_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden2_size, num_classes)
        self.act_output = nn.Sigmoid()
    #training function
    def forward(self,x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.act_output(out)
        return out
    
#Instantiate class with input, hidden layer, and output size 
model2 = Layer2(152,1024,16384,18211)
#Change to double data type
model2.double()



Layer2(
  (fc1): Linear(in_features=152, out_features=1024, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=1024, out_features=16384, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=16384, out_features=18211, bias=True)
  (act_output): Sigmoid()
)

In [8]:
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model1.parameters(), lr = 0.001)

In [9]:
# compute accuracy (no_grad is optional)
def deviation():
    with torch.no_grad():
        y_pred = model1(valIn_t)

    accuracy = (abs((y_pred-valOut_t)).sum()/valOut_t.sum())
    RSS = ((y_pred-valOut_t)**2).float().sum()
    return (f"Deviation: {100*accuracy}%    RSS: {RSS}")



In [10]:
#training loop 
def train(epoch_number, batch_size):
    for epoch in range(epoch_number):
        for i in range(0, len(trainIn_t), batch_size):
            Xbatch = trainIn_t[i:i+batch_size]
            y_pred = model1(Xbatch)
            ybatch = trainOut_t[i:i+batch_size]
            loss = loss_fn(y_pred, ybatch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f"Finished epoch {epoch}, {deviation()}")


In [11]:

#Train model (epochs, mini batch size)
train(10,20)


Finished epoch 0, Deviation: 17.892881898534544%    RSS: 15555.869140625
Finished epoch 1, Deviation: 11.48338010366795%    RSS: 10693.8583984375
Finished epoch 2, Deviation: 10.46596781410086%    RSS: 9818.3876953125
Finished epoch 3, Deviation: 10.464663280569644%    RSS: 9590.61328125
Finished epoch 4, Deviation: 10.479883107307346%    RSS: 9477.072265625
Finished epoch 5, Deviation: 10.508274772395866%    RSS: 9422.1279296875
Finished epoch 6, Deviation: 10.534529131345181%    RSS: 9390.927734375
Finished epoch 7, Deviation: 10.559538460233485%    RSS: 9369.6748046875
Finished epoch 8, Deviation: 10.576420810035707%    RSS: 9346.037109375
Finished epoch 9, Deviation: 10.59567591556908%    RSS: 9318.140625


In [12]:
# compute accuracy on training set 
def deviationtrain():
    with torch.no_grad():
        y_pred = model1(trainIn_t)

    accuracy = (abs((y_pred-trainOut_t)).sum()/trainOut_t.sum())
    RSS = ((y_pred-trainOut_t)**2).float().sum()
    return (f"Deviation: {100*accuracy}%    RSS: {RSS}")

deviationtrain()

'Deviation: 8.199492882304247%    RSS: 26294.015625'

In [13]:
def hot_encode(cell,compound):
    cell_vec = np.zeros(cell_mapping.size)
    cell_dict = {value: index for index, value in enumerate(cell_mapping)}

    compound_vec = np.zeros(compound_mapping.size)
    compound_dict = {value: index for index, value in enumerate(compound_mapping)}


    cell_vec[cell_dict[cell]]=1
    compound_vec[compound_dict[compound]]=1
    vector = np.concatenate((cell_vec, compound_vec), axis = 0)
    tensor = torch.from_numpy(vector)
    return tensor


In [14]:
model1(hot_encode("NK cells", "Clotrimazole"))

tensor([0.1438, 0.2501, 0.4922,  ..., 0.4484, 0.4482, 0.5629],
       dtype=torch.float64, grad_fn=<SigmoidBackward0>)

In [18]:
def get_expression(cell_type, compound_name):
    tensor = model1(hot_encode(cell_type,compound_name))
    np_array = tensor.detach().numpy()
    df = pd.DataFrame(np_array)
    return unnormalize(df[0],norm_min,norm_max)


df = get_expression("NK cells", "Clotrimazole")
print(df)




0        0.386862
1        0.557180
2       -0.933238
3       -0.130187
4        0.684468
           ...   
18206    0.462209
18207    0.126884
18208    0.255395
18209   -0.459017
18210   -0.116373
Length: 18211, dtype: float64
